In [ ]:
!pip install datasets
!pip install transformers

In [114]:
# Importing required libraries

from datasets import load_dataset
from transformers import AutoTokenizer
import numpy as np
import tensorflow as tf
from transformers import TFAutoModelForQuestionAnswering
from transformers import create_optimizer
from bs4 import BeautifulSoup
import requests

In [ ]:
# Loading squad dataset

squad = load_dataset("squad", split="train[:20000]")

data = squad.train_test_split(test_size=0.2)

In [115]:
# Importing and initializing tokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
max_length = 400  # The maximum length of a feature.
doc_stride = 120  # overlap between two part of the context.

In [ ]:
# Function to truncate and map the start and end tokens of the answer to the context.

def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        stride=doc_stride,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
# Mapping data to the function.

tokenized_datasets = data.map(
    preprocess_function, batched=True, remove_columns=data["train"].column_names)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [ ]:
# Loading the pretrained model for fine tuning.

model = TFAutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForQuestionAnswering: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it

In [ ]:
# Assigning parameter values

learning_rate = 2e-4
num_train_epochs = 10
batch_size = 8

In [ ]:
# Prepare our train and test data.

train_set = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    shuffle=True,
    batch_size=batch_size,)

validation_set = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    shuffle=False,
    batch_size=batch_size,)

In [ ]:
# Creating our optimizer

total_train_steps = len(train_set) * num_train_epochs

optimizer, schedule = create_optimizer(init_lr=learning_rate, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
# Compile the model.

model.compile(optimizer=optimizer, jit_compile=True, metrics=["accuracy"])

In [ ]:
# Train the model.

model.fit(
    train_set,
    validation_data=validation_set,
    epochs=num_train_epochs
)

Epoch 1/10
2000/2000 [==============================] - 767s 365ms/step - loss: 1.8616 - end_logits_accuracy: 0.5166 - start_logits_accuracy: 0.4949 - val_loss: 1.1846 - val_end_logits_accuracy: 0.6740 - val_start_logits_accuracy: 0.6352
Epoch 2/10
2000/2000 [==============================] - 736s 368ms/step - loss: 0.9832 - end_logits_accuracy: 0.7278 - start_logits_accuracy: 0.7009 - val_loss: 1.1589 - val_end_logits_accuracy: 0.6945 - val_start_logits_accuracy: 0.6628
Epoch 3/10
2000/2000 [==============================] - 703s 351ms/step - loss: 0.6212 - end_logits_accuracy: 0.8133 - start_logits_accuracy: 0.7951 - val_loss: 1.2377 - val_end_logits_accuracy: 0.6773 - val_start_logits_accuracy: 0.6547
Epoch 4/10
2000/2000 [==============================] - 742s 371ms/step - loss: 0.3976 - end_logits_accuracy: 0.8770 - start_logits_accuracy: 0.8643 - val_loss: 1.3580 - val_end_logits_accuracy: 0.6885 - val_start_logits_accuracy: 0.6575
Epoch 5/10
2000/2000 [==========================

In [67]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Saving the model.

model.save_pretrained('/content/drive/MyDrive/Exported_models/my_QA_model/my_model')

In [116]:
# Load the saved model

mymodel = TFAutoModelForQuestionAnswering.from_pretrained("/content/drive/MyDrive/Exported_models/my_QA_model/my_model")

All model checkpoint layers were used when initializing TFDistilBertForQuestionAnswering.

All the layers of TFDistilBertForQuestionAnswering were initialized from the model checkpoint at /content/drive/MyDrive/Exported_models/my_QA_model/my_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForQuestionAnswering for predictions without further training.


In [178]:
# Taking input from user

inp= input("Enter url: ")

Enter url: https://blog.datagran.io/posts/whats-ai-ml-and-dl-what-are-the-differences


In [179]:
# fetching page data and response.

page= requests.get(inp)
page

<Response [200]>

In [180]:
# Processing the data

soup= BeautifulSoup(page.content)
full_context= str(soup.text.strip())
full_context= full_context.replace('\n','')

In [181]:
full_context

"What’s AI, ML and DL? What are the differences?THE BLOGTHE BLOGCATEGORY:Artificial IntelligenceTAGS:Artificial IntelligenceMachine LearningWhat’s AI, ML and DL? What are the differences?Necati DemirChief AI OfficerIt’s very common for a lot of people who jump into the world of AI to get drown in the terms and abbreviations. Artificial Intelligence? Machine Learning? Deep Learning? AI? ML? DL? … and many more. In this short article, Here is my overview about those keywords.\u200dArtificial Intelligence (AI)\u200dLet’s start with artificial intelligence (AI). I always prefer to describe AI as an umbrella term which covers everything in this world. It is not a technical method or a name of a specific algorithm.\xa0 AI is a research field in computer science that focuses on developing methods which can perform tasks that a human can accomplish.\xa0The term artificial intelligence was first coined by John McCarthy in 1956 and his definition was “the science and engineering of making intell

In [182]:
# Creating chunks of context if context size is too large for tokenizer.

big= False
if len(full_context) > 500:
  K= 6
  splt_char='.'
  temp = full_context.split(splt_char)
  context_list= [splt_char.join(temp[i:i+K]) for i in range(0, len(temp), K)][:-1]
  big = True

In [177]:
# Going through each context chunk and extracting answer/response.

def my_answer(context_list, question, tokenizer, mymodel):
  temp=4

  for context in context_list:

    start_position=0
    end_position=0

    inputs = tokenizer([question], [context], return_tensors="np")
    outputs = mymodel(inputs)

    start_position = np.argmax(outputs.start_logits[0])
    end_position = np.argmax(outputs.end_logits[0])

    if start_position<=end_position and outputs.start_logits[0][start_position]>temp:

      start_position_final= start_position
      end_position_final= end_position
      inputs_final= inputs
      temp= outputs.start_logits[0][start_position]

    else:
      pass

  # Decoding tokens to form answers.
  answer = inputs_final["input_ids"][0, start_position_final: end_position_final + 1]
  return tokenizer.decode(answer)

In [188]:
# Get answer
def get_answer(big):
  if big:
    print(my_answer(context_list, question, tokenizer, mymodel))
  else:
    inputs = tokenizer([question], [full_context], return_tensors="np")
    outputs = mymodel(inputs)
    start_position = np.argmax(outputs.start_logits[0])
    end_position = np.argmax(outputs.end_logits[0])
    answer = inputs["input_ids"][0, start_position: end_position + 1]
    print(tokenizer.decode(answer))

In [183]:
# defining question

question= input("Enter your question: ")

Enter your question: Who coined the term machine learning?


In [186]:
get_answer(big)

arthur samuel


In [187]:
# defining another question

question= "What is deep learning?"

get_answer(big)

creating multiple layers of neural networks
